# Test 1: Tables vs Views: join distinct chrom, pos, ref, alt from cg and illumina tables for chromosome 1

## Querying Tables

### Create tables of distinct chrom, pos, ref, alt 

– create table of distinct chrom, pos, ref, alt from illumina   

    create table p7 product.illumina distinct   
    (pos int, ref string, alt string )   
    partitioned by (chrom string, pos block int)  
    
– create table of distinct chrom, pos, ref, alt from cg   

    create table p7 product.comgen distinct  
    (pos int, ref string, alt string )   
    partitioned by (chrom string, pos block int)  

### Join tables

-- join variants  
    select count(1) from (  
    
        SELECT distinct CAST(coalesce(t0.pos, t1.pos) AS int) AS pos,  
            coalesce(t0.ref, t1.ref) AS ref,  
            coalesce(t0.alt, t1.alt) AS alt,  
            coalesce(t0.chrom, t1.chrom) AS chrom  
        FROM p7_product.illumina_distinct t0  
        FULL OUTER JOIN p7_product.comgen_distinct t1  
            ON t0.chrom = t1.chrom  
            AND t0.pos = t1.pos  
            AND t0.ref = t1.ref  
            AND t0.alt = t1.alt  
        WHERE t0.chrom = ’1’  
        AND t1.chrom = ’1’  
    ) as test;  

### Explain Plan

Estimated Per-Host Requirements: Memory=3.83GB VCores=2  


10:AGGREGATE [FINALIZE]  
|  output: count:merge(1)  
|  
09:EXCHANGE [UNPARTITIONED]  
|  
04:AGGREGATE  
|  output: count(1)  
|  
08:AGGREGATE [FINALIZE]  
|  group by: CAST(coalesce(t0.pos, t1.pos) AS INT), coalesce(t0.ref, t1.ref), coalesce(t0.alt, t1.alt),   coalesce(t0.chrom, t1.chrom)  
|  
07:EXCHANGE [HASH(CAST(coalesce(t0.pos, t1.pos) AS INT),coalesce(t0.ref, t1.ref),coalesce(t0.alt,   t1.alt),coalesce(t0.chrom, t1.chrom))]  
|  
03:AGGREGATE  
|  group by: CAST(coalesce(t0.pos, t1.pos) AS INT), coalesce(t0.ref, t1.ref), coalesce(t0.alt, t1.alt),   coalesce(t0.chrom, t1.chrom)  
|  
02:HASH JOIN [FULL OUTER JOIN, PARTITIONED]  
|  hash predicates: t1.chrom = t0.chrom, t1.pos = t0.pos, t1.ref = t0.ref, t1.alt = t0.alt  
|  other predicates: t0.chrom = '1', t1.chrom = '1'  
|  
|--06:EXCHANGE [HASH(t0.chrom,t0.pos,t0.ref,t0.alt)]  
|  |  
|  00:SCAN HDFS [p7_product.illumina_distinct t0]  
|     partitions=230/2916 files=230 size=210.14MB  
|  
05:EXCHANGE [HASH(t1.chrom,t1.pos,t1.ref,t1.alt)]  
|  
01:SCAN HDFS [p7_product.comgen_distinct t1]  
   partitions=230/2916 files=460 size=326.15MB  

Result 1: Fetched 1 row(s) in 6.03s  
Result 2: Fetched 1 row(s) in 6.15s  
Result 3: Fetched 1 row(s) in 5.98s  
Average time: 6.05s  

## Querying Views

-- create view of distinct chrom, pos, ref, alt from illumina  

    create view p7_product.illumina_view as  
        (select * from p7_product.illumina_distinct)  
        
-- create table of distinct chrom, pos, ref, alt from cg  

    create view p7_product.comgen_view as  
        (select * from p7_product.comgen_distinct)

select count(1) from (    

    SELECT distinct CAST(coalesce(t0.pos, t1.pos) AS int) AS pos,  
        coalesce(t0.ref, t1.ref) AS ref,   
        coalesce(t0.alt, t1.alt) AS alt,  
        coalesce(t0.chrom, t1.chrom) AS chrom  
    FROM p7_product.illumina_view t0  
    FULL OUTER JOIN p7_product.comgen_view t1  
        ON t0.chrom = t1.chrom  
        AND t0.pos = t1.pos  
        AND t0.ref = t1.ref  
        AND t0.alt = t1.alt  
    WHERE t0.chrom = ’1’  
    AND t1.chrom = ’1’  
    ) as test;

Result 1: Fetched 1 row(s) in 5.89s  
Result 2: Fetched 1 row(s) in 6.20s  
Result 3: Fetched 1 row(s) in 5.92s  
Average time: 6.00s  

# Test 2: Querying a View that includes a Join

-- create the view

    create view p7_product.test_view as (
    SELECT distinct CAST(coalesce(t0.pos, t1.pos) AS int) AS pos,  
        coalesce(t0.ref, t1.ref) AS ref,  
        coalesce(t0.alt, t1.alt) AS alt,  
        coalesce(t0.chrom, t1.chrom) AS chrom  
    FROM p7_product.illumina_distinct t0  
    FULL OUTER JOIN p7_product.comgen_distinct t1  
        ON t0.chrom = t1.chrom   
        AND t0.pos = t1.pos  
        AND t0.ref = t1.ref  
        AND t0.alt = t1.alt;  
 
-- query the view  

    select count(1) from (  
        select *  
        from p7_product.test_view  
        WHERE chrom = ’1’  
    )  
    as test;

### Explain Plan

Estimated Per-Host Requirements: Memory=65.27GB VCores=2  
    
08:EXCHANGE [UNPARTITIONED]    
|  
07:AGGREGATE [FINALIZE]  
|  group by: CAST(coalesce(t0.pos, t1.pos) AS INT), coalesce(t0.ref, t1.ref), coalesce(t0.alt, t1.alt), coalesce(t0.chrom, t1.chrom)  
|  
06:EXCHANGE [HASH(CAST(coalesce(t0.pos, t1.pos) AS INT),coalesce(t0.ref, t1.ref),coalesce(t0.alt, t1.alt),coalesce(t0.chrom, t1.chrom))]  
|  
03:AGGREGATE  
|  group by: CAST(coalesce(t0.pos, t1.pos) AS INT), coalesce(t0.ref, t1.ref), coalesce(t0.alt, t1.alt), coalesce(t0.chrom, t1.chrom)  
|  
02:HASH JOIN [FULL OUTER JOIN, PARTITIONED]  
|  hash predicates: t1.chrom = t0.chrom, t1.pos = t0.pos, t1.ref = t0.ref, t1.alt = t0.alt  
|  
|--05:EXCHANGE [HASH(t0.chrom,t0.pos,t0.ref,t0.alt)]  
|  |  
|  00:SCAN HDFS [p7_product.illumina_distinct t0]  
|     partitions=2916/2916 files=2928 size=2.67GB  
|  
04:EXCHANGE [HASH(t1.chrom,t1.pos,t1.ref,t1.alt)]  
|  
01:SCAN HDFS [p7_product.comgen_distinct t1]  
   partitions=2916/2916 files=5828 size=4.12GB  

Result 1: Fetched 1 row(s) in 132.55s  
Result 2: Fetched 1 row(s) in 130.79s  
Result 3: Fetched 1 row(s) in 137.28s  
Average Time: 133.54s  

# Test 3: Creating View using WHERE clause

    create view p7_product.test_view_chr1 as  
    
         SELECT distinct CAST(coalesce(t0.pos, t1.pos) AS int) AS pos,  
              coalesce(t0.ref, t1.ref) AS ref,  
              coalesce(t0.alt, t1.alt) AS alt,  
              coalesce(t0.chrom, t1.chrom) AS chrom  
         FROM p7_product.illumina_distinct t0  
         FULL OUTER JOIN p7_product.comgen_distinct t1  
             ON t0.chrom = t1.chrom  
             AND t0.pos = t1.pos  
             AND t0.ref = t1.ref  
             AND t0.alt = t1.alt  
        WHERE t0.chrom = ’1’  
        AND t1.chrom = ’1’;   
        
    select count(1) from (  
        select *  
        from p7_product.test_view_chr1  
    )
    as test;  

Estimated Per-Host Requirements: Memory=65.27GB VCores=2  
  
08:EXCHANGE [UNPARTITIONED]  
|  
07:AGGREGATE [FINALIZE]  
|  group by: CAST(coalesce(t0.pos, t1.pos) AS INT), coalesce(t0.ref, t1.ref), coalesce(t0.alt, t1.alt),   coalesce(t0.chrom, t1.chrom)  
|  
06:EXCHANGE [HASH(CAST(coalesce(t0.pos, t1.pos) AS INT),coalesce(t0.ref, t1.ref),coalesce(t0.alt,   t1.alt),coalesce(t0.chrom, t1.chrom))]  
|  
03:AGGREGATE  
|  group by: CAST(coalesce(t0.pos, t1.pos) AS INT), coalesce(t0.ref, t1.ref), coalesce(t0.alt, t1.alt),   coalesce(t0.chrom, t1.chrom)  
|  
02:HASH JOIN [FULL OUTER JOIN, PARTITIONED]  
|  hash predicates: t1.chrom = t0.chrom, t1.pos = t0.pos, t1.ref = t0.ref, t1.alt = t0.alt  
|  
|--05:EXCHANGE [HASH(t0.chrom,t0.pos,t0.ref,t0.alt)]  
|  |  
|  00:SCAN HDFS [p7_product.illumina_distinct t0]  
|     partitions=2916/2916 files=2928 size=2.67GB  
|  
04:EXCHANGE [HASH(t1.chrom,t1.pos,t1.ref,t1.alt)]  
|  
01:SCAN HDFS [p7_product.comgen_distinct t1]  
   partitions=2916/2916 files=5828 size=4.12GB  

Result 1: Fetched 1 row(s) in 10.98s  
Result 2: Fetched 1 row(s) in 9.99s  
Result 3: Fetched 1 row(s) in 9.18s  
Average Time: 10.05s  